# CWRU Scenario2 디버깅 노트북 (Varying Load)

이 노트북은 CWRU의 분할/로더/학습 파이프라인을 단계적으로 실행하여 1.0 정확도 반복 문제를 재현하고 원인을 분리합니다.

- 목적: 동일 파일 내 윈도우 분할로 인한 분포 유사성/누수 여부 진단
- 기준: 파일 교집합(train∩val, train∩test, val∩test)=Ø, 윈도우 인덱스 분리 확인
- 중단 조건: 첫 도메인(0HP) 정확도 < 80%면 이후 도메인 중단



In [1]:
# 환경/임포트
import os, sys, json, random, math, time
from pathlib import Path
import numpy as np
import torch

project_root = Path('/data/home/kyj2024/TextVibCLIP')
os.chdir(project_root)
sys.path.insert(0, str(project_root))

from src.data_loader import BearingDataset, create_domain_dataloaders
from src.data_cache import create_cached_domain_dataloaders, create_cached_first_domain_dataloader, clear_all_caches
from src.continual_trainer import ContinualTrainer
from configs.model_config import DATA_CONFIG, CWRU_DATA_CONFIG

print('cwd=', os.getcwd())
print('CUDA=', torch.cuda.is_available())
print('DATA_CONFIG=', DATA_CONFIG)
print('CWRU_DATA_CONFIG=', CWRU_DATA_CONFIG)



/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/data/home/kyj2024/anaconda3/envs/TVCLIP

cwd= /data/home/kyj2024/TextVibCLIP
CUDA= True
DATA_CONFIG= {'data_dir': 'data_scenario1', 'dataset_type': 'uos', 'domain_order': [600, 800, 1000, 1200, 1400, 1600], 'validation_split': 0.2, 'test_split': 0.2, 'signal_normalization': 'standardize', 'window_size': 2048, 'overlap_ratio': 0.5, 'max_text_length': 128}
CWRU_DATA_CONFIG= {'data_dir': 'data_scenario2', 'dataset_type': 'cwru', 'domain_order': [0, 1, 2, 3], 'validation_split': 0.2, 'test_split': 0.2, 'signal_normalization': 'standardize', 'window_size': 2048, 'overlap_ratio': 0.8, 'max_text_length': 128}


In [2]:
# CWRU 파일 나열
from glob import glob

data_dir = Path(CWRU_DATA_CONFIG['data_dir'])
assert data_dir.exists(), f"{data_dir} 가 존재하지 않습니다. prepare_cwru_no_leakage.py 실행 필요"

# 하위 폴더 확인
try:
    subdirs = sorted([p.name for p in data_dir.iterdir() if p.is_dir()])
    print('하위 폴더:', subdirs)
except Exception as e:
    print('하위 폴더 나열 실패:', e)

for hp in [0,1,2,3]:
    pattern = data_dir / f"Load_{hp}hp" / '*.mat'
    files = sorted(glob(str(pattern)))
    print(f"{hp}HP: {len(files)} files (pattern={pattern})")
    print([Path(f).name for f in files][:10])



하위 폴더: ['Load_0hp', 'Load_1hp', 'Load_2hp', 'Load_3hp']
0HP: 16 files (pattern=data_scenario2/Load_0hp/*.mat)
['B_0hp_test_01.mat', 'B_0hp_train_01.mat', 'B_0hp_train_05.mat', 'B_0hp_train_09.mat', 'H_0hp_train_02.mat', 'IR_0hp_test_01.mat', 'IR_0hp_train_01.mat', 'IR_0hp_train_05.mat', 'IR_0hp_train_09.mat', 'OR_0hp_test_04.mat']
1HP: 16 files (pattern=data_scenario2/Load_1hp/*.mat)
['B_1hp_test_02.mat', 'B_1hp_train_02.mat', 'B_1hp_train_06.mat', 'B_1hp_val_01.mat', 'H_1hp_val_01.mat', 'IR_1hp_test_02.mat', 'IR_1hp_train_02.mat', 'IR_1hp_train_06.mat', 'IR_1hp_val_01.mat', 'OR_1hp_test_01.mat']
2HP: 16 files (pattern=data_scenario2/Load_2hp/*.mat)
['B_2hp_test_03.mat', 'B_2hp_train_03.mat', 'B_2hp_train_07.mat', 'B_2hp_val_02.mat', 'H_2hp_test_01.mat', 'IR_2hp_test_03.mat', 'IR_2hp_train_03.mat', 'IR_2hp_train_07.mat', 'IR_2hp_val_02.mat', 'OR_2hp_test_02.mat']
3HP: 16 files (pattern=data_scenario2/Load_3hp/*.mat)
['B_3hp_test_04.mat', 'B_3hp_train_04.mat', 'B_3hp_train_08.mat', 'B_3

In [3]:
# 현재 로더 분할 검증: 도메인별 train/val/test 파일셋 동일성 확인
from collections import defaultdict

# 도메인 0HP 기준으로 로더 생성 (빠르게 검증)
loaders = create_domain_dataloaders(
    data_dir=str(data_dir),
    domain_order=[0],
    dataset_type='cwru',
    batch_size=4,
    num_workers=0,
    use_collate_fn=False
)

train_ds = loaders[0]['train'].dataset
val_ds = loaders[0]['val'].dataset
test_ds = loaders[0]['test'].dataset

# 각 subset이 참조하는 파일 목록 수집
train_files = [Path(p).name for p in train_ds.file_paths]
val_files = [Path(p).name for p in val_ds.file_paths]
test_files = [Path(p).name for p in test_ds.file_paths]

print('파일 수:', len(train_files), len(val_files), len(test_files))
print('train==val==test 동일 여부:', train_files==val_files==test_files)
print('train∩val:', len(set(train_files)&set(val_files)))
print('train∩test:', len(set(train_files)&set(test_files)))
print('val∩test:', len(set(val_files)&set(test_files)))

# 윈도우 분할 범위 확인
print('train window range:', getattr(train_ds, '_window_split_range', None))
print('val window range:', getattr(val_ds, '_window_split_range', None))
print('test window range:', getattr(test_ds, '_window_split_range', None))



파일 수: 11 2 3
train==val==test 동일 여부: False
train∩val: 0
train∩test: 0
val∩test: 0
train window range: (0.0, 1.0)
val window range: (0.0, 1.0)
test window range: (0.0, 1.0)


In [4]:
# 누수 정량 체크: 동일 파일에서 추출된 윈도우 인덱스 분포 확인
# __getitem__ 내부 랜덤 인덱스 캐싱 동작에 의존하므로, 일부 샘플을 직접 뽑아 파일별 인덱스 구간을 관찰

import itertools

sample_indices = [0, len(train_ds)//3, 2*len(train_ds)//3]

# 각 subset에서 특정 파일을 기준으로 어떤 윈도우 인덱스 범위를 쓰는지 대략적 확인
# (정밀 재현은 소스 수정이 필요하므로 여기선 관찰 중심)

def peek_samples(ds, k=5):
    xs = []
    for i in np.linspace(0, len(ds)-1, k, dtype=int):
        x = ds[i]  # tuple 반환 가정
        xs.append(x)
    return xs

print('샘플 관찰(train/val/test)...')
_ = peek_samples(train_ds, k=5)
_ = peek_samples(val_ds, k=5)
_ = peek_samples(test_ds, k=5)
print('완료')



샘플 관찰(train/val/test)...
완료


In [5]:
# 파일 단위 분리 전략(모의) 설계: 동일 도메인 내 파일을 disjoint하게 3분할
from glob import glob
from sklearn.model_selection import train_test_split

# 올바른 CWRU 폴더명은 'Load_0hp'
cand_patterns = [
    data_dir / 'Load_0hp' / '*.mat',
    data_dir / '0hp' / '*.mat',
    data_dir / '0HP' / '*.mat',
]
all_files = []
for pat in cand_patterns:
    files = sorted(glob(str(pat)))
    if files:
        all_files = files
        print(f'사용 경로: {pat} -> {len(files)} files')
        break

if not all_files:
    print('경로에 .mat 파일이 없습니다. 디렉토리 구조를 출력합니다.')
    print('data_dir=', data_dir)
    print('하위 폴더:', sorted([p.name for p in data_dir.iterdir() if p.is_dir()]))
    raise FileNotFoundError('CWRU 0HP 파일을 찾을 수 없습니다. 예상: data_scenario2/Load_0hp/*.mat')

# 파일 분할
train_files, tmp = train_test_split(all_files, test_size=0.4, random_state=42, shuffle=True)
val_files, test_files = train_test_split(tmp, test_size=0.5, random_state=42, shuffle=True)

print(len(train_files), len(val_files), len(test_files))
print('교집합 검사:', len(set(train_files)&set(val_files)), len(set(train_files)&set(test_files)), len(set(val_files)&set(test_files)))



사용 경로: data_scenario2/Load_0hp/*.mat -> 16 files
9 3 4
교집합 검사: 0 0 0


In [6]:
# 모의 분할을 적용한 커스텀 Dataset 래퍼(간단)
from torch.utils.data import Dataset

class FileSubsetWrapper(Dataset):
    def __init__(self, base: BearingDataset, allowed_filenames):
        self.base = base
        self.allowed = set(Path(f).name for f in allowed_filenames)
        # 필터: base.file_paths 와 metadata_list 동기화
        keep = [i for i, p in enumerate(base.file_paths) if Path(p).name in self.allowed]
        self.file_paths = [base.file_paths[i] for i in keep]
        self.metadata_list = [base.metadata_list[i] for i in keep]
        # 길이/윈도우 계산은 base 로직 사용. 필요 시 base의 window 범위 그대로 사용.
        self.subset = base.subset
        self.dataset_type = base.dataset_type
        self.domain_value = base.domain_value
        self._window_split_range = getattr(base, '_window_split_range', None)
        self._window_split_type = getattr(base, '_window_split_type', None)
        self._random_window_indices = getattr(base, '_random_window_indices', {})
    def __len__(self):
        return len(self.base)  # 근사치; 정확한 길이 계산은 base 내부 의존이라 여기선 샘플 검증 위주
    def __getitem__(self, idx):
        return self.base[idx]

# 0HP용 원본 dataset 3개 생성
base_loaders = create_domain_dataloaders(str(data_dir), [0], 'cwru', batch_size=4, num_workers=0, use_collate_fn=False)
base_train = base_loaders[0]['train'].dataset
base_val = base_loaders[0]['val'].dataset
base_test = base_loaders[0]['test'].dataset

# 래퍼 적용: 파일 교집합 없는 분리 강제
train_wrapped = FileSubsetWrapper(base_train, train_files)
val_wrapped = FileSubsetWrapper(base_val, val_files)
test_wrapped = FileSubsetWrapper(base_test, test_files)

print('wrapped files:', len(train_wrapped.file_paths), len(val_wrapped.file_paths), len(test_wrapped.file_paths))
print('교집합 다시 검사:',
      len(set(Path(p).name for p in train_wrapped.file_paths) & set(Path(p).name for p in val_wrapped.file_paths)),
      len(set(Path(p).name for p in train_wrapped.file_paths) & set(Path(p).name for p in test_wrapped.file_paths)),
      len(set(Path(p).name for p in val_wrapped.file_paths) & set(Path(p).name for p in test_wrapped.file_paths)))



wrapped files: 7 1 2
교집합 다시 검사: 0 0 0


In [7]:
# (선택) 0HP 한 도메인만 간이 학습으로 정확도 sanity-check (80% 미만이면 중단)
from torch.utils.data import DataLoader

simple_train_loader = DataLoader(base_train, batch_size=4, shuffle=True, num_workers=0)
simple_val_loader = DataLoader(base_val, batch_size=4, shuffle=False, num_workers=0)

# 프로젝트의 전체 모델/트레이너를 쓰기보단, 기존 트레이너의 first_domain만 빠르게 호출
trainer = ContinualTrainer(device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                           data_dir=str(data_dir), dataset_type='cwru', domain_order=[0])
trainer.batch_size = 4

first_results = trainer.train_first_domain(first_domain_dataloader=simple_train_loader, num_epochs=2)
# 노트북에서는 훈련 후 평가 함수가 내부 호출된다고 가정 (로그로 확인)

